In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import torch
import torch.nn as nn
import torch.optim as optim
import ot
import sys
import os
import tarfile
import scipy.io
import h5py
import gzip
from scipy.sparse import issparse
from scipy.sparse import csr_matrix

In [14]:
adata = sc.read_h5ad("/workspace/ImputationOT/data/sim1_norm.h5ad")
print(adata)

AnnData object with n_obs × n_vars = 12097 × 9979
    obs: 'Cell', 'Batch', 'Group', 'ExpLibSize', 'sum', 'detected', 'percent_top_50', 'percent_top_100', 'percent_top_200', 'percent_top_500', 'total', 'Discard', 'size_factors'
    var: 'Gene', 'BaseGeneMean', 'OutlierFactor', 'GeneMean', 'BatchFacBatch1', 'BatchFacBatch2', 'BatchFacBatch3', 'BatchFacBatch4', 'BatchFacBatch5', 'BatchFacBatch6', 'DEFacGroup1', 'DEFacGroup2', 'DEFacGroup3', 'DEFacGroup4', 'DEFacGroup5', 'DEFacGroup6', 'DEFacGroup7', 'mean', 'detected'
    layers: 'counts'


In [20]:
print(adata.obs['Batch'].value_counts())

Batch
Batch1    2908
Batch2    2422
Batch3    2120
Batch4    1929
Batch5    1761
Batch6     957
Name: count, dtype: int64


In [23]:
print(adata.obs['Group'].value_counts())

Group
Group1    3688
Group2    2885
Group3    1929
Group4    1546
Group5    1089
Group6     756
Group7     204
Name: count, dtype: int64


In [22]:
X = adata.X
print("Matrix Shape:", X.shape)
# print("Density:", np.count_nonzero(X) / (X.shape[0] * X.shape[1]))
print("Density:", X.nnz / (X.shape[0] * X.shape[1]))
print("Minimum Value:", X.min())
print("Maximum Value:", X.max())

Matrix Shape: (12097, 9979)
Density: 0.5690072737107684
Minimum Value: 0.0
Maximum Value: 6.739929826089321
